# Fine-tuning a model with the Trainer API

Transformers, veri kümeniz üzerinde sağladığı önceden eğitilmiş modellerden herhangi birine ince ayar yapmanıza yardımcı olmak için bir Trainer sınıfı sağlar. Son bölümde tüm veri ön işleme çalışmalarını yaptıktan sonra, Trainer'ı tanımlamak için sadece birkaç adımınız kaldı. En zor kısım, CPU üzerinde çok yavaş çalışacağı için Trainer.train()'i çalıştıracak ortamı hazırlamak olacaktır. Eğer bir GPU kurulumunuz yoksa, Google Colab üzerinden ücretsiz GPU'lara veya TPU'lara erişim sağlayabilirsiniz.

Aşağıdaki kod örnekleri, önceki bölümdeki örnekleri zaten uyguladığınızı varsaymaktadır. Burada neye ihtiyacınız olduğunu özetleyen kısa bir özet bulunmaktadır:

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-08-07 06:35:25.222035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 06:35:25.222159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 06:35:25.350885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

# Training

Eğiticimizi tanımlamadan önceki ilk adım, Eğiticinin eğitim ve değerlendirme için kullanacağı tüm hiperparametreleri içerecek bir TrainingArguments sınıfı tanımlamaktır. Sağlamanız gereken tek argüman, eğitilen modelin kaydedileceği bir dizin ve yol boyunca kontrol noktalarıdır. Geri kalan her şey için, temel bir ince ayar için oldukça iyi çalışması gereken varsayılanları bırakabilirsiniz.

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

İkinci adım modelimizi tanımlamaktır. Önceki bölümde olduğu gibi, iki etiketli AutoModelForSequenceClassification sınıfını kullanacağız:

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2
).to("cuda")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Bölüm 2'den farklı olarak, bu ön eğitimli modeli örneklendirdikten sonra bir uyarı aldığınızı fark edeceksiniz. Bunun nedeni, BERT'in cümle çiftlerini sınıflandırma konusunda ön eğitime tabi tutulmamış olmasıdır, bu nedenle ön eğitime tabi tutulmuş modelin başı atılmış ve yerine dizi sınıflandırmasına uygun yeni bir baş eklenmiştir. Uyarılar, bazı ağırlıkların kullanılmadığını (atılan ön eğitim kafasına karşılık gelenler) ve bazılarının rastgele başlatıldığını (yeni kafa için olanlar) gösterir. Sonunda sizi modeli eğitmeye teşvik eder ki şimdi yapacağımız da tam olarak budur.

Modelimizi oluşturduktan sonra, şimdiye kadar oluşturulmuş tüm nesneleri (model, training_args, eğitim ve doğrulama veri kümeleri, data_collator ve tokenizer) ona aktararak bir Trainer tanımlayabiliriz:

In [4]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Burada yaptığımız gibi tokenizer'ı geçtiğinizde, Trainer tarafından kullanılan varsayılan data_collator'ın daha önce tanımlandığı gibi bir DataCollatorWithPadding olacağını unutmayın, bu nedenle bu çağrıda data_collator=data_collator satırını atlayabilirsiniz. Bölüm 2'de işlemin bu kısmını göstermek yine de önemliydi!

Veri kümemiz üzerinde modele ince ayar yapmak için Trainer'ımızın train() yöntemini çağırmamız yeterlidir:

In [5]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.528900
1000,0.300300


TrainOutput(global_step=1377, training_loss=0.34513849993936374, metrics={'train_runtime': 171.2448, 'train_samples_per_second': 64.259, 'train_steps_per_second': 8.041, 'total_flos': 405114969714960.0, 'train_loss': 0.34513849993936374, 'epoch': 3.0})

Bu, ince ayarı başlatacak (GPU'da birkaç dakika sürecektir) ve her 500 adımda bir eğitim kaybını rapor edecektir. Ancak modelinizin ne kadar iyi (veya kötü) performans gösterdiğini size söylemez. Bunun nedeni şudur:

Evaluation_strategy'yi "steps" (her eval_steps'te değerlendirme) ya da "epoch" (her epoch'un sonunda değerlendirme) olarak ayarlayarak Eğiticiye eğitim sırasında değerlendirme yapmasını söylemedik. Söz konusu değerlendirme sırasında bir metrik hesaplaması için Eğiticiye bir compute_metrics() işlevi sağlamadık (aksi takdirde değerlendirme sadece kaybı yazdırırdı, ki bu çok sezgisel bir sayı değildir).

## Evaluation

Şimdi kullanışlı bir **compute_metrics()** fonksiyonunu nasıl oluşturabileceğimizi ve bir sonraki eğitimimizde nasıl kullanabileceğimizi görelim. Fonksiyon bir **EvalPrediction nesnesi (predictions alanı ve label_ids alanı olan adlandırılmış bir tuple)** almalı ve dizeleri kayan değerlere eşleyen bir sözlük döndürmelidir (dizeler döndürülen metriklerin adları, kayan değerler ise değerleridir). Modelimizden bazı tahminler almak için **Trainer.predict()** komutunu kullanabiliriz:

In [6]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


predict() yönteminin çıktısı, üç alana sahip başka bir adlandırılmış tuple'dır: predictions, label_ids ve metrics. Metrics alanı, aktarılan veri kümesindeki kaybın yanı sıra bazı zaman metriklerini (toplamda ve ortalama olarak tahmin etmenin ne kadar sürdüğü) içerecektir. compute_metrics() fonksiyonumuzu tamamlayıp Trainer'a ilettiğimizde, bu alan compute_metrics() tarafından döndürülen metrikleri de içerecektir.

Gördüğünüz gibi, tahminler 408 x 2 şeklinde iki boyutlu bir dizidir (408, kullandığımız veri kümesindeki öğe sayısıdır). Bunlar, predict() işlevine aktardığımız veri kümesinin her bir öğesi için logitlerdir (önceki bölümde gördüğünüz gibi, tüm Transformer modelleri logit döndürür). Bunları etiketlerimizle karşılaştırabileceğimiz tahminlere dönüştürmek için, ikinci eksende maksimum değere sahip indeksi almamız gerekir:

In [7]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
preds

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,

In [8]:
!pip install -q evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8651960784313726, 'f1': 0.9056603773584906}

Model kafasının rastgele başlatılması elde ettiği metrikleri değiştirebileceğinden, elde ettiğiniz kesin sonuçlar değişebilir. Burada, modelimizin doğrulama kümesinde %85,78'lik bir doğruluğa ve 89,97'lik bir F1 skoruna sahip olduğunu görebiliriz. Bunlar, GLUE kıyaslaması için MRPC veri kümesindeki sonuçları değerlendirmek için kullanılan iki metriktir. BERT makalesindeki tabloda temel model için 88,9 F1 skoru bildirilmiştir. Biz şu anda daha iyi sonucu açıklayan cased modelini kullanırken bu model uncased modeldi.

Her şeyi bir araya getirerek `compute_metrics()` fonksiyonumuzu elde ediyoruz:

In [10]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Ve her epoch'un sonunda metrikleri raporlamak için kullanıldığını görmek için, bu `compute_metrics()` işleviyle yeni bir `Trainer`'ı nasıl tanımladığımızı burada görebilirsiniz:

In [11]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2
).to("cuda")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation_strategy'si "epoch" olarak ayarlanmış yeni bir TrainingArguments ve yeni bir model oluşturduğumuza dikkat edin - aksi takdirde, zaten eğittiğimiz modelin eğitimine devam etmiş oluruz. Yeni bir eğitim çalıştırması başlatmak için şunu yürütürüz:

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.420780,0.821078,0.872600
2,0.569500,0.450426,0.813725,0.871622
3,0.348200,0.716702,0.828431,0.884868


TrainOutput(global_step=1377, training_loss=0.38946196889219376, metrics={'train_runtime': 134.9616, 'train_samples_per_second': 81.534, 'train_steps_per_second': 10.203, 'total_flos': 405114969714960.0, 'train_loss': 0.38946196889219376, 'epoch': 3.0})

Bu kez, eğitim kaybının yanı sıra her epoğun sonunda doğrulama kaybını ve ölçümlerini de rapor edecektir. Yine, ulaştığınız kesin Accuracy/F1 score, modelin rastgele kafa başlatması nedeniyle bizim bulduğumuzdan biraz farklı olabilir, ancak aynı top sahasında olmalıdır.

Eğitmen, birden fazla GPU veya TPU'da kutudan çıktığı gibi çalışacak ve karışık hassasiyetli eğitim gibi birçok seçenek sunacaktır (eğitim argümanlarınızda fp16 = True kullanın). Bölüm 10'da desteklediği her şeyin üzerinden geçeceğiz.

Bu, Trainer API kullanarak ince ayar yapmaya giriş bölümünü sonlandırmaktadır. Bunu en yaygın NLP görevleri için yapmanın bir örneği Bölüm 7'de verilecektir, ancak şimdilik aynı şeyi saf PyTorch'ta nasıl yapacağımıza bakalım.